In [ ]:
# This work is copyright (c) the Regents of the University of Minnesota, 2017.  
# It was created and last updated by Kelly Thompson, 2018-06-27.

# We anticipate adding an open license to this software, pending administrative approval, but in the meantime,
# please just contact us to ask.  Thanks!

In [ ]:
import requests
import urllib3
from lxml import etree
import pandas as pd
from copy import deepcopy

In [ ]:
from datetime import date
today = str(date.today())

### Call the Alma Analytics API to retrieve the GOBI holdings reports

In [ ]:
url_root = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/analytics/reports'
key = 'REDACTED'
limit = '1000'

In [ ]:
paths = ['/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI electronic holdings 1990-1999 - ids not excluded',
         '/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI electronic holdings 2000-2009 - ids not excluded',
         '/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI electronic holdings 2010-2019 - ids not excluded',
         '/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/Duluth GOBI electronic holdings 1990-2019 - ids not excluded',
         '/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/Duluth GOBI physical holdings 1990-2019 pub dates - full',
         '/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI physical holdings 1990-1994 pub dates - full',
         '/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI physical holdings 1995-1999 pub dates - full',
         '/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI physical holdings 2000-2004 pub dates - full',
         '/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI physical holdings 2005-2009 pub dates - full',
         '/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI physical holdings 2010-2019 pub dates - full'
        ]


In [ ]:
outputs = ['TC_GOBI_electronic_holdings_1990-1999_ids_not_excluded',
           'TC_GOBI_electronic_holdings_2000-2009_ids_not_excluded',
           'TC_GOBI_electronic_holdings_2010-2019_ids_not_excluded',
           'Duluth_GOBI_electronic_holdings_1990-2019_full',
           'Duluth_GOBI_physical_holdings_1990-2019_pub_dates_full',
           'TC_GOBI_physical_holdings_1990-1994_pub_dates_full',
           'TC_GOBI_physical_holdings_1995-1999_pub_dates_full',
           'TC_GOBI_physical_holdings_2000-2004_pub_dates_full',
           'TC_GOBI_physical_holdings_2005-2009_pub_dates_full',
           'TC_GOBI_physical_holdings_2010-2019_pub_dates_full'
          ]

In [ ]:
def fetch_report (path, url_root, key, limit):
    params2 = {'apikey': key, 'path': path, 'limit': limit}
    r = requests.get(url_root, params = params2)
    print(r.url)
    print(r.status_code)
    if r.status_code != 200:
        print(r.text)
    return r

In [ ]:
# revised

def get_text(r, index, url_root, key, limit):
    
    
    print("Chunk 1")
    
    root = etree.fromstring(r.content)
    #print(etree.dump(root))
    
    resultset = root.findall("./QueryResult/ResultXml/*/*")
    print("number of results " + str(len(resultset)))
    
    a = etree.Element('response')
    #etree.dump(a)
    resultset.pop(0)
    for each in resultset:
        a.append(deepcopy(each))
    b = a.getchildren()
    print(len(b))
    
    print("Chunk 2")
    
    f = root.find("./QueryResult/IsFinished")
    if f.text == "false":
        token = root.find("./QueryResult/ResumptionToken")
        
    while (f.text == "false"):

        if token.text != None:
            params3 = {'apikey': key, 'token': token.text}
            r2 = requests.get(url_root, params = params3 )
            print(r2.url)

            root2 = etree.fromstring(r2.content)
            resultset2 = root2.findall("./QueryResult/ResultXml/*/*")
            print(len(resultset2))

            for each in resultset2:
                a.append(deepcopy(each))
            c = a.getchildren()
            print(len(c))

            f = root2.find("./QueryResult/IsFinished")
            
            if f is None:
                
                print(token.text)
                
    else:
        print("done looping resumption tokens")
            
    print("Chunk 4")

    all_records = []
    rows = a.getchildren()
    print(len(rows))
    for each in rows:
        record = {}
        gcs = each.getchildren()
        for column in gcs:
            record[column.tag] = column.text
        all_records.append(record)
    data = pd.DataFrame(all_records)

    print("Chunk 5")

    output_file = outputs[index] + '-' + today + '.csv'
    print(output_file)
    data.to_csv(output_file)
    print("Success!")
    print()

In [ ]:
'''
FOR TESTING ONLY
paths = [#'/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI electronic holdings 1990-1999 - ids not excluded',
         #'/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI electronic holdings 2000-2009 - ids not excluded',
         #'/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI electronic holdings 2010-2019 - ids not excluded',
         #'/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/Duluth GOBI electronic holdings 1990-2019 - ids not excluded',
         #'/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/Duluth GOBI physical holdings 1990-2019 pub dates - full',
        #'/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI physical holdings 1990-1994 pub dates - full',
        #'/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI physical holdings 1995-1999 pub dates - full',
         #'/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI physical holdings 1990-1999 pub dates - full',
         #'/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/drafts/TC GOBI physical holdings 2000-09 pub dates - full',
        #'/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI physical holdings 2000-2004 pub dates - full',
        #'/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI physical holdings 2005-2009 pub dates - full'
         '/shared/University of Minnesota/Twin Cities/Kelly Thompson/GOBI holdings/TC GOBI physical holdings 2010-2019 pub dates - full'
        ]

outputs = [#'TC_GOBI_electronic_holdings_1990-1999_ids_not_excluded',
           #'TC_GOBI_electronic_holdings_2000-2009_ids_not_excluded',
           #'TC_GOBI_electronic_holdings_2010-2019_ids_not_excluded',
           #'Duluth_GOBI_electronic_holdings_1990-2019_full',
           #'Duluth_GOBI_physical_holdings_1990-2019_pub_dates_full',
           #'TC_GOBI_physical_holdings_1990-1999_pub_dates_full',
            #'TC_GOBI_physical_holdings_1990-1994_pub_dates_full',
            #'TC_GOBI_physical_holdings_1995-1999_pub_dates_full'
           #'TC_GOBI_physical_holdings_2000-09_pub_dates_full',
            #'TC_GOBI_physical_holdings_2000-2004_pub_dates_full',
            #'TC_GOBI_physical_holdings_2005-2009_pub_dates_full',
           'TC_GOBI_physical_holdings_2010-2019_pub_dates_full',
           
          ]
'''

In [ ]:
paths

In [ ]:
import timeit

In [ ]:
num_of_tries = []
index = 0
for path in paths:
    i = 1
    print("try " + str(i))
    r = fetch_report(path, url_root, key, limit)
    while r.status_code != 200:
        i += 1
        print("try " + str(i))
        r = fetch_report(path, url_root, key, limit)
    print("DONE")
    num_of_tries.append(i)
    print(num_of_tries)
    
    %timeit -n1 -r1 -c get_text(r, index, url_root, key, limit)
    
    index += 1
    print()

### Now we should have all of our reports, which we can process to keep only the data we need, in the format required.

In [ ]:
import os
import re
folder = "2018-06-26"
# get lists of files
tce_files = [f for f in os.listdir(folder) if re.match('.*TC.*.elec.*.csv', f)]
tce_files

In [ ]:
tcp_files = [f for f in os.listdir(folder) if re.match('.*TC.*.phys.*.csv', f)]
tcp_files

In [ ]:
dule_files = [f for f in os.listdir(folder) if re.match('Dul.*.elec.*.csv', f)]
dule_files

In [ ]:
dulp_files = [f for f in os.listdir(folder) if re.match('Dul.*.phys.*.csv', f)]
dulp_files

In [ ]:
# read in all TC electronic reports from Analytics
tce = pd.DataFrame()
print(tce)
for f in tce_files:
    tce_new = pd.read_csv(folder + "\\" + f, sep = ",", encoding = "utf-8", dtype = str,
                          usecols=['{urn:schemas-microsoft-com:xml-analysis:rowset}Column11',
                                   '{urn:schemas-microsoft-com:xml-analysis:rowset}Column6'])
    tce = pd.concat([tce, tce_new], ignore_index=True)
tce.rename(columns={"{urn:schemas-microsoft-com:xml-analysis:rowset}Column11": "ISBN RegEx",
                    "{urn:schemas-microsoft-com:xml-analysis:rowset}Column6": "ElectronicCollection"}, inplace=True)
tce['Library Code 10 char'] = 'TCEBOOK'
tce['YBP sub-account'] = str('195099')
tce

In [ ]:
tc_ecolls_to_exclude = pd.read_csv('IDs to exclude\elec colls to exclude\TC electronic colls to exclude.txt',
                                   sep='\t', dtype=str)
tc_ecolls_to_exclude

In [ ]:
tce2 = tce[~tce['ElectronicCollection'].isin(tc_ecolls_to_exclude['ecoll'])]
tce2

In [ ]:
# read in all TC physical reports from Analytics
tcp = pd.DataFrame()
print(tcp)
for f in tcp_files:
    tcp_new = pd.read_csv(folder + "\\" + f, sep = ",", encoding = "utf-8", dtype = str,
                          usecols=['{urn:schemas-microsoft-com:xml-analysis:rowset}Column7',
                                   '{urn:schemas-microsoft-com:xml-analysis:rowset}Column8',
                                   '{urn:schemas-microsoft-com:xml-analysis:rowset}Column6'])
    tcp = pd.concat([tcp, tcp_new], ignore_index=True)
tcp.rename(columns={"{urn:schemas-microsoft-com:xml-analysis:rowset}Column7" : "ISBN RegEx", 
                    "{urn:schemas-microsoft-com:xml-analysis:rowset}Column8" : "Library Code 10 char", 
                    "{urn:schemas-microsoft-com:xml-analysis:rowset}Column6" : "YBP sub-account"}, 
           inplace=True)
tcp

In [ ]:
tce2.columns

In [ ]:
tce2.drop(columns=['ElectronicCollection'],inplace=True)
tce2

In [ ]:
tcp.columns

In [ ]:
# concatenate tc physcial and electronic dataframes into one
tc = pd.concat([tce2, tcp], ignore_index=True)
tc

In [ ]:
# read in all Duluth electronic reports from Analytics
dule = pd.DataFrame()
print(dule)
for f in dule_files:
    dule_new = pd.read_csv(folder + "\\" + f, sep = ",", encoding = "utf-8", dtype = str,
                          usecols=['{urn:schemas-microsoft-com:xml-analysis:rowset}Column11',
                                   '{urn:schemas-microsoft-com:xml-analysis:rowset}Column6'])
    dule = pd.concat([dule, dule_new], ignore_index=True)
dule.rename(columns={"{urn:schemas-microsoft-com:xml-analysis:rowset}Column11": "ISBN RegEx",
                     "{urn:schemas-microsoft-com:xml-analysis:rowset}Column6": "ElectronicCollection"}, 
            inplace=True)
dule['Library Code 10 char'] = 'DULEBOOK'
dule['YBP sub-account'] = str('583999')
dule

In [ ]:
dul_ecolls_to_exclude = pd.read_csv('IDs to exclude\elec colls to exclude\Duluth electronic colls to exclude.txt',
                                   sep='\t', dtype=str)
dul_ecolls_to_exclude

In [ ]:
dule2 = dule[~dule['ElectronicCollection'].isin(dul_ecolls_to_exclude['ecoll'])]
dule2

In [ ]:
# read in all TC physical reports from Analytics
dulp = pd.DataFrame()
print(dulp)
for f in dulp_files:
    dulp_new = pd.read_csv(folder + "\\" + f, sep = ",", encoding = "utf-8", dtype = str,
                          usecols=['{urn:schemas-microsoft-com:xml-analysis:rowset}Column7',
                                   '{urn:schemas-microsoft-com:xml-analysis:rowset}Column8',
                                   '{urn:schemas-microsoft-com:xml-analysis:rowset}Column6'])
    dulp = pd.concat([dulp, dulp_new], ignore_index=True)
dulp.rename(columns={"{urn:schemas-microsoft-com:xml-analysis:rowset}Column7" : "ISBN RegEx", 
                    "{urn:schemas-microsoft-com:xml-analysis:rowset}Column8" : "Library Code 10 char", 
                    "{urn:schemas-microsoft-com:xml-analysis:rowset}Column6" : "YBP sub-account"}, 
           inplace=True)
dulp

In [ ]:
dule2.columns

In [ ]:
dule2.drop(columns=['ElectronicCollection'],inplace=True)
dule2

In [ ]:
dulp.columns

In [ ]:
# concatenate tc physcial and electronic dataframes into one
dul = pd.concat([dule2, dulp], ignore_index=True)
dul

In [ ]:
################################
# remove invalid TC ISBNs from set

In [ ]:
tc_isbns = tc['ISBN RegEx']
print(tc_isbns.size)

In [ ]:
tc.drop_duplicates(inplace=True)
tc

In [ ]:
tc_isbns = tc['ISBN RegEx']
print(tc_isbns.size)

In [ ]:
bad_tc_isbns = pd.read_table("C:/Users/kjthomps/Documents/GOBI holdings reports/IDs to exclude/TC and Law invalid ISBNs.txt")
bad_tc_isbns = bad_tc_isbns['ISBN']
print(bad_tc_isbns.size)

In [ ]:
tc1 = tc[~tc['ISBN RegEx'].isin(bad_tc_isbns)]
tc1['ISBN RegEx'].size

In [ ]:
bad_tc_e_isbns = pd.read_table("C:/Users/kjthomps/Documents/GOBI holdings reports/IDs to exclude/TC electronic ISBNs to exclude.txt")
bad_tc_e_isbns = bad_tc_e_isbns['ISBN']
bad_tc_e_isbns.size

In [ ]:
tc2 = tc1[~tc1['ISBN RegEx'].isin(bad_tc_e_isbns)]
tc2['ISBN RegEx'].size

In [ ]:
tc_final = tc2.rename(columns={'ISBN RegEx':'ISBN'})
tc_final.to_csv(folder + "\\" + 'TC-all.txt', sep="\t", index=False)

In [ ]:
################################
# create law file and fix law account number

law = tc_final
law['YBP sub-account'].replace("195099", "590099", inplace= True)
law

In [ ]:
law.to_csv(folder + "\\" + "law-all.txt", sep="\t", index=False)

In [ ]:
####################################
# remove invalid duluth isbns

dul_isbns = dul['ISBN RegEx']
dul_isbns.size

In [ ]:
dul.drop_duplicates(inplace=True)
dul

In [ ]:
bad_dul_isbns = pd.read_table("C:/Users/kjthomps/Documents/GOBI holdings reports/IDs to exclude/Duluth invalid ISBNs.txt")
bad_dul_isbns = bad_dul_isbns['ISBN']
bad_dul_isbns.size

In [ ]:
dul1 = dul[~dul['ISBN RegEx'].isin(bad_dul_isbns)]
dul1['ISBN RegEx'].size

In [ ]:
dul_final = dul1.rename(columns={'ISBN RegEx':'ISBN'})
dul_final

In [ ]:
dul_final.to_csv(folder + "\\" + "Duluth-all.txt", sep="\t", index = False)

### Now that we have the current month's inventory, we need to find the intersections with the previous inventory, to produce files of adds and deletes.

In [ ]:
old_folder = folder + "\\last batch"
# get lists of files
old_TC_file = [f for f in os.listdir(old_folder) if re.match('.*TC.*', f)][0]
old_TC_file = old_folder + "\\" + old_TC_file
old_TC_file

In [ ]:
new_TC_file = folder + "\TC-all.txt"
new_TC_file

In [ ]:
TC_adds = "TC-adds.txt"
TC_deletes = "TC-deletes.txt"

In [ ]:
law_adds_file = "LAW-adds.txt"
law_deletes_file = "LAW-deletes.txt"

In [ ]:
old_DUL_file = [f for f in os.listdir(old_folder) if re.match('.*Dul.*', f)][0]
old_DUL_file = old_folder + "\\" + old_DUL_file
old_DUL_file

In [ ]:
new_DUL_file = folder + "\Duluth-all.txt"
new_DUL_file

In [ ]:
dul_adds = "DUL-adds.txt"
dul_deletes = "DUL-deletes.txt"

In [ ]:
def find_adds_dels (old_file, new_file):
    
    old_df = pd.read_table(old_file)
    old_df = old_df.drop_duplicates()
    
    print(old_df.columns)
    
    new_df = pd.read_table(new_file)
    new_df = new_df.drop_duplicates()
    
    print(new_df.columns)
    
    old_df = old_df.rename(index=str, columns={"ISBN": "ISBN_old", "Library Code 10 char": "LibraryCode_old", "YBP sub-account" : "YBPacct_old"})
    
    new_df = new_df.rename(index=str, columns={"ISBN": "ISBN_new", "Library Code 10 char": "LibraryCode_new", "YBP sub-account" : "YBPacct_new"})
    
    old_merged = pd.merge(left = old_df, right = new_df, how='left', left_on=['ISBN_old','LibraryCode_old'], right_on=['ISBN_new','LibraryCode_new'])
    deleted = old_merged[pd.isnull(old_merged['LibraryCode_new'])]
    deleted = deleted.loc[:,['ISBN_old','LibraryCode_old','YBPacct_old']]
    
    
    new_merged = pd.merge(left = old_df, right = new_df, how='right', left_on=['ISBN_old','LibraryCode_old'], right_on=['ISBN_new','LibraryCode_new'])
    added = new_merged[pd.isnull(new_merged['LibraryCode_old'])]
    added = added.loc[:,['ISBN_new','LibraryCode_new','YBPacct_new']]
    
    deletes = pd.merge(left=old_df, right=new_df, how = "left", left_on=['ISBN_old','LibraryCode_old'], 
                       right_on=['ISBN_new','LibraryCode_new'])
    deletes = deletes[pd.isnull(deletes['LibraryCode_new'])]
    deletes = deletes.loc[:,['ISBN_old','LibraryCode_old','YBPacct_old']]
    
    adds = pd.merge(left=old_df, right=new_df, how = "right", left_on=['ISBN_old','LibraryCode_old'], 
                       right_on=['ISBN_new','LibraryCode_new'])
    adds = adds[pd.isnull(adds['LibraryCode_old'])]
    adds = adds.loc[:,['ISBN_new','LibraryCode_new','YBPacct_new']]
    
    adds = adds.rename(index=str, columns={"ISBN_new" : "ISBN", "LibraryCode_new" : "Library Code 10 char", 
                                                             "YBPacct_new" : "YBP sub-account"})
    
    deletes = deletes.rename(index=str, columns={"ISBN_old" : "ISBN", "LibraryCode_old" : "Library Code 10 char", 
                                                             "YBPacct_old" : "YBP sub-account"})
    
    return adds, deletes

In [ ]:
adds, deletes = find_adds_dels(old_TC_file, new_TC_file)

In [ ]:
adds

In [ ]:
deletes

In [ ]:
adds.to_csv(folder + "\\" + TC_adds, sep="\t", index = False)
deletes.to_csv(folder + "\\" + TC_deletes, sep="\t", index = False)

In [ ]:
adds["YBP sub-account"].replace(195099, 590099, inplace= True)
adds

In [ ]:
adds.to_csv(folder + "\\" + law_adds_file, sep="\t", index = False)

In [ ]:
deletes["YBP sub-account"].replace(195099, 590099, inplace= True)
deletes

In [ ]:
deletes.to_csv(folder + "\\" + law_deletes_file, sep="\t", index = False)

In [ ]:
adds, deletes = find_adds_dels(old_DUL_file, new_DUL_file)

In [ ]:
adds

In [ ]:
deletes

In [ ]:
adds.to_csv(folder + "\\" + dul_adds, sep="\t", index = False)
deletes.to_csv(folder + "\\" + dul_deletes, sep="\t", index = False)